In [176]:
import pandas as pd

In [215]:
df = pd.read_csv('nba_player_data_2015.csv')

In [304]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [305]:
df.head()

,+/1,3P,3PA,AST,BLK,Date,FG,FGA,FT,FTA,...,TOV,TRB,Team,minutes,name,opp_score,started,team_score,is_Home,Team_Win
0,16.0,2.0,3.0,5.0,0.0,10/28/2014,2.0,6.0,3.0,6.0,...,2.0,5.0,Houston,30:55,Patrick Beverley,90,1,108,0,1
1,12.0,3.0,6.0,6.0,0.0,10/28/2014,7.0,17.0,15.0,16.0,...,0.0,1.0,Houston,30:46,James Harden,90,1,108,0,1
2,14.0,0.0,1.0,3.0,2.0,10/28/2014,8.0,16.0,0.0,1.0,...,2.0,13.0,Houston,30:31,Terrence Jones,90,1,108,0,1
3,11.0,5.0,8.0,2.0,0.0,10/28/2014,5.0,10.0,1.0,1.0,...,2.0,3.0,Houston,29:34,Trevor Ariza,90,1,108,0,1
4,20.0,0.0,0.0,1.0,1.0,10/28/2014,3.0,5.0,7.0,16.0,...,1.0,11.0,Houston,20:34,Dwight Howard,90,1,108,0,1


In [306]:
df['spread'] = df['team_score']-df['opp_score']

In [307]:
df.columns

Index([u'+/1', u'3P', u'3PA', u'AST', u'BLK', u'Date', u'FG', u'FGA', u'FT',
       u'FTA', u'Game_Id', u'ORB', u'Opponent', u'PF', u'PTS', u'STL', u'TOV',
       u'TRB', u'Team', u'minutes', u'name', u'opp_score', u'started',
       u'team_score', u'is_Home', u'Team_Win', u'spread'],
      dtype='object')

In [308]:
groups= df.groupby(['Game_Id','Team'],sort=False).sum()

In [296]:
groups.head()

,,+/1,3P,3PA,AST,BLK,FG,FGA,FT,FTA,ORB,PF,PTS,STL,TOV,TRB,opp_score,started,team_score,spread
Game_Id,Team,,,,,,,,,,,,,,,,,,,
10/28/2014_LA Lakers_Houston,Houston,90.0,12.0,29.0,22.0,3.0,31.0,73.0,34.0,50.0,14.0,30.0,108.0,7.0,13.0,47.0,1170,5.0,1404,234
10/28/2014_Houston_LA Lakers,LA Lakers,-90.0,3.0,10.0,16.0,3.0,28.0,79.0,31.0,39.0,11.0,32.0,90.0,7.0,11.0,36.0,1404,5.0,1170,-234
10/28/2014_New Orleans_Orlando,Orlando,-85.0,4.0,11.0,17.0,9.0,32.0,84.0,16.0,21.0,16.0,25.0,84.0,5.0,18.0,56.0,1313,5.0,1092,-221
10/28/2014_Orlando_New Orleans,New Orleans,85.0,4.0,17.0,20.0,17.0,41.0,101.0,15.0,31.0,26.0,17.0,101.0,10.0,9.0,62.0,1092,5.0,1313,221
10/28/2014_San Antonio_Dallas,Dallas,-5.0,8.0,21.0,17.0,3.0,38.0,78.0,16.0,19.0,9.0,20.0,100.0,9.0,10.0,33.0,1313,5.0,1300,-13


In [309]:
matrix = groups.as_matrix()

In [310]:
ids = []
for game in df['Game_Id'].unique():
    ids.append(game+ '%aa'+game.split('_')[1])
    ids.append(game+ '%aa'+game.split('_')[2])
ids[0:10]

['10/28/2014_LA Lakers_Houston%aaLA Lakers',
 '10/28/2014_LA Lakers_Houston%aaHouston',
 '10/28/2014_New Orleans_Orlando%aaNew Orleans',
 '10/28/2014_New Orleans_Orlando%aaOrlando',
 '10/28/2014_San Antonio_Dallas%aaSan Antonio',
 '10/28/2014_San Antonio_Dallas%aaDallas',
 '10/29/2014_Boston_Brooklyn%aaBoston',
 '10/29/2014_Boston_Brooklyn%aaBrooklyn',
 '10/29/2014_Charlotte_Milwaukee%aaCharlotte',
 '10/29/2014_Charlotte_Milwaukee%aaMilwaukee']

In [311]:
len(matrix)

2460

In [312]:
new_frame = pd.DataFrame(matrix, columns = groups.columns, index = ids)

In [313]:
team_data = new_frame[['3P','3PA', 'AST', 'BLK', 'FG', 'FGA', 'FT', 'FTA', 'ORB',
       'PF', 'PTS', 'STL', 'TOV', 'TRB','is_Home','Team_Win']]

In [314]:
team_data = team_data.reset_index()

In [315]:
team_data['Team'] = team_data['index'].apply(lambda x: x.split('%aa')[1])

In [316]:
team_data['index'] = team_data['index'].apply(lambda x: x.split('%aa')[0])

In [317]:
team_data['is_Home'] = team_data['is_Home'].apply(lambda x: x>0).astype(int)
team_data['Team_Win'] = team_data['Team_Win'].apply(lambda x: x>0).astype(int)

In [318]:
team_data.head()

,index,3P,3PA,AST,BLK,FG,FGA,FT,FTA,ORB,PF,PTS,STL,TOV,TRB,is_Home,Team_Win,Team
0,10/28/2014_LA Lakers_Houston,12.0,29.0,22.0,3.0,31.0,73.0,34.0,50.0,14.0,30.0,108.0,7.0,13.0,47.0,0,1,LA Lakers
1,10/28/2014_LA Lakers_Houston,3.0,10.0,16.0,3.0,28.0,79.0,31.0,39.0,11.0,32.0,90.0,7.0,11.0,36.0,1,0,Houston
2,10/28/2014_New Orleans_Orlando,4.0,11.0,17.0,9.0,32.0,84.0,16.0,21.0,16.0,25.0,84.0,5.0,18.0,56.0,0,0,New Orleans
3,10/28/2014_New Orleans_Orlando,4.0,17.0,20.0,17.0,41.0,101.0,15.0,31.0,26.0,17.0,101.0,10.0,9.0,62.0,1,1,Orlando
4,10/28/2014_San Antonio_Dallas,8.0,21.0,17.0,3.0,38.0,78.0,16.0,19.0,9.0,20.0,100.0,9.0,10.0,33.0,0,0,San Antonio


In [319]:
both_teams_total = team_data.groupby('index',sort=False).sum().reset_index()
##doing weird things. 
weird = pd.merge(both_teams_total,team_data,on='index')

In [320]:
#fill in opponent data in an insufficient way
team_data['opp_3P'] = weird['3P_x']-team_data['3P']
team_data['opp_3PA'] = weird['3PA_x']-team_data['3PA']
team_data['opp_AST'] = weird['AST_x']-team_data['AST']
team_data['opp_BLK'] = weird['BLK_x']-team_data['BLK']
team_data['opp_FG'] = weird['FG_x']-team_data['FG']
team_data['opp_FGA'] = weird['FGA_x']-team_data['FGA']
team_data['opp_FT'] = weird['FT_x']-team_data['FT']
team_data['opp_FTA'] = weird['FTA_x']-team_data['FTA']
team_data['opp_PTS'] = weird['PTS_x']-team_data['PTS']
team_data['opp_ORB'] = weird['ORB_x']-team_data['ORB']
team_data['opp_STL'] = weird['STL_x']-team_data['STL']
team_data['opp_PF'] = weird['PF_x']-team_data['PF']
team_data['opp_TOV'] = weird['TOV_x']-team_data['TOV']

In [321]:
def opponent(row):
    if row['Team'] == row['index'].split('_')[1]:
        return row['index'].split('_')[2]
    else:
        return row['index'].split('_')[1]

In [322]:
team_data['opponent'] =team_data.apply(opponent, axis=1)

In [323]:
team_data['Game_Id'] = team_data['index']
team_data.drop('index',axis=1, inplace=True)

In [324]:
##get rid of games that are playoffs
team_data = team_data.iloc[:2460]
team_data.to_csv('team_boxscores_2014.csv')

In [258]:
df = pd.read_csv('nba_player_data_2015.csv')
df = df.iloc[:31430]
df.to_csv('correct_nba_player_data_2015.csv')